In [2]:
import csv as csv
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
import matplotlib.pyplot as plt
import re
import warnings
import seaborn as sns
from pylab import mpl  
mpl.rcParams['font.sans-serif'] = ['SimHei'] 

# 读取数据集
df = pd.read_csv('cy1.csv')  #训练集

col = ['content', 'label']
df = df[col]

cat=[]
for i in range(len(df['label'])):
    if df['label'][i]==1:
        cat.append('阐述')
    if df['label'][i]==2:
        cat.append('推理')
    if df['label'][i]==3:
        cat.append('判断')
    if df['label'][i]==4:
        cat.append('方法')
df['cat']=cat

cat_label_df = df[['cat','label']].drop_duplicates().sort_values('label').reset_index(drop=True)
cat_to_label = dict(cat_label_df.values)
label_to_cat = dict(cat_label_df[['label', 'cat']].values)

d = {'cat':df['cat'].value_counts().index, 'count': df['cat'].value_counts()}
df_cat = pd.DataFrame(data=d).reset_index(drop=True)

d = {'cat':df['cat'].value_counts().index, 'count': df['cat'].value_counts()}
df_cat = pd.DataFrame(data=d).reset_index(drop=True)
df_cat

,cat,count
0,阐述,341
1,方法,83
2,判断,55
3,推理,21


In [3]:
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("stop_words.txt")

df['content'] = df['content'].map(lambda x: ' '.join(jieba.cut(x))) #以空格分隔jieba分词结果

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MAC\AppData\Local\Temp\jieba.cache
Loading model cost 1.093 seconds.
Prefix dict has been built succesfully.


In [4]:
content=df['content']

In [5]:
content_value=content.values.reshape(-1, 1)

In [6]:
# content_value.shape

(500, 1)

In [12]:
#使用imblearn进行随机过采样
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(content_value, df.label.values)

In [13]:
from collections import Counter
Counter(y_resampled)

Counter({1: 341, 2: 341, 3: 341, 4: 341})

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
 
count_vect = CountVectorizer()
content_counts = count_vect.fit_transform(content)

In [7]:
content_counts

<500x1690 sparse matrix of type '<class 'numpy.int64'>'
	with 6219 stored elements in Compressed Sparse Row format>

In [8]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_sample(content_counts, df.label.values)

In [9]:
from collections import Counter
Counter(y_resampled)

Counter({1: 69, 2: 301, 3: 205, 4: 212})

In [10]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_sample(content_counts, df.label.values)

In [11]:
from collections import Counter
Counter(y_resampled)

Counter({1: 340, 2: 341, 3: 340, 4: 341})

In [13]:
y_resampled

array([1, 1, 2, ..., 4, 4, 4], dtype=int64)

In [15]:
m=X_resampled.todense()

In [16]:
m

matrix([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [23]:
# 引入必要的库
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

# 加载数据
iris = datasets.load_iris()
X = iris.data
y = iris.target

# # 将标签二值化
# y = label_binarize(y, classes=[0, 1, 2])
# # 设置种类
# n_classes = y.shape[1]

In [26]:
y[0]

0